In [55]:
import nest_asyncio
nest_asyncio.apply()
from tensorflow.keras import layers
import tensorflow as tf
import numpy as np
from gym_derk.envs import DerkEnv
from src.environments import EnvSix
from tensorflow import keras
from tensorflow.keras import Sequential, layers
import asyncio

In [56]:
class FightEnv(DerkEnv):
    def __init__(self, left_team, right_team, *args, **kwargs):
        self.left_team = left_team
        self.right_team = right_team
        super().__init__(*args, **kwargs, n_arenas=1)
    
    def prepoc(self, action):
        print(action)
        real_action = []
        real_action.append(action[0])
        real_action.append(action[1])
        real_action.append(max(action[2], 0))

        spells = self.pick(action[3:7])
        focus = self.pick(action[7:])
        real_action.append(spells)
        real_action.append(focus)
        return real_action
    
    def pick(self, odds: np.ndarray):
        odds[odds<0] = 0
        odds = odds + 0.001
        normalized_odds = np.array(odds) / sum(odds)
        normalized_odds /= normalized_odds.sum()

        return np.random.choice(len(odds), p=normalized_odds)
    
    def step(self, actions):
        print(actions)
        actions = actions.reshape(6, 15)
        processed_actions = np.apply_along_axis(self.prepoc, -1, actions)
        resultats = asyncio.get_event_loop().run_until_complete(self.async_step(processed_actions))
        return resultats[0]
    
    def fight(self):
        obs = self.reset()
        for i in range(100):
            left_team_action = np.array([self.left_team[i](obs[i]) for i in range(3)])
            right_team_action = np.array([self.right_team[i](obs[i+3]) for i in range(3)])
            actions = np.concatenate((left_team_action, right_team_action))
            print("\n")
            print(actions)
            print("\n")
            obs = self.step(actions)
        

In [57]:
def left_test(obs):
    return np.array([0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1 ,0], dtype=np.float32)

def right_test(obs):
    return np.random.uniform(-1, 1, 15)

In [58]:
left_team = [left_test for i in range(3)]
right_team = [right_test for i in range(3)]

In [ ]:
env.close()
env = FightEnv(left_team, right_team)

In [60]:
env.fight()



[[ 0.          0.          1.          0.          0.          0.
   1.          0.          0.          0.          0.          0.
   0.          1.          0.        ]
 [ 0.          0.          1.          0.          0.          0.
   1.          0.          0.          0.          0.          0.
   0.          1.          0.        ]
 [ 0.          0.          1.          0.          0.          0.
   1.          0.          0.          0.          0.          0.
   0.          1.          0.        ]
 [ 0.3155103  -0.60360877 -0.2500903   0.2976753   0.0480715   0.85437916
  -0.63013973 -0.82405624 -0.41773399 -0.49447267  0.91379791  0.67292189
  -0.40358338  0.45057721  0.39719162]
 [ 0.80287537 -0.66627337  0.00265405 -0.38943399  0.15854365  0.81927582
  -0.51002346  0.30401715  0.42340302  0.31426407 -0.91511279  0.51482347
   0.81675679 -0.73893899 -0.0572876 ]
 [ 0.85755259  0.47188395  0.47285884  0.91570652 -0.77476516  0.27761073
  -0.14185706  0.99947541  0.79302469

KeyboardInterrupt: 